In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#INPUT
df = pd.read_csv(r'input.csv')
min_conf = 0.5
min_sup = 0.5
quasifiIden = ['age', 'gender', 'education', 'marital-status', 'native-country', 'race' ]
k = 5


In [3]:
#FIND RULE FROM DATASET D BY USING APRIORI


#data cleaning
for lab, row in df.iterrows():
    for cell in row:
        if cell == ' ?':
            df.drop(lab, inplace=True)
            break
# initializing encoded dataframe
encoded_df = df.copy(deep=True)
column = len(df.columns)
for i in df.columns:
  for j in list(set(df[i])):
    column = len(encoded_df.columns)
    encoded_df.insert(column,i +':'+str(j), False)

# transform category data to binary
column = len(df.columns)
column_arr = encoded_df.columns
i = 0
for lab, row in df.iterrows():
  j = 0
  for j in range(9):
    getValueOfCell = df.columns[j]+ ':' + str(df.iloc[i][j])
    index = next((n for n, item in enumerate(column_arr) if item == getValueOfCell), -1)
    encoded_df.iloc[i, index] = True
  i += 1

for i in df.columns:
  encoded_df.drop(i, axis=1, inplace=True)

frequent_itemsets = apriori(encoded_df, min_support=min_sup, use_colnames = True)

rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = min_conf)
rules = rules[['antecedents', 'consequents', 'support', 'confidence']]
rules.to_csv('rules.csv', index=False) #SET OF RULE CARE


C:\Users\Omen\AppData\Local\Temp\ipykernel_8764\3557504809.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df.insert(column,i +':'+str(j), False)
C:\Users\Omen\AppData\Local\Temp\ipykernel_8764\3557504809.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df.insert(column,i +':'+str(j), False)
C:\Users\Omen\AppData\Local\Temp\ipykernel_8764\3557504809.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [ ]:
# filter output must have quasify attribute in the rules
final_res = rules[['antecedents', 'consequents', 'support', 'confidence']]

final_res.insert(4, 'HaveQuasifi', False)
final_res.insert(5, 'QIrhs', False)


for i in range(len(final_res)):
  for ele in final_res['antecedents'][i]:
    for item in quasifiIden:
      if item in ele:
        final_res['HaveQuasifi'][i] = True
        break

  for ele in final_res['consequents'][i]:
    for item in quasifiIden:
      if item in ele:
        final_res['HaveQuasifi'][i] = True
        final_res['QIrhs'][i] = True
        break

# Rcare
final_res = final_res[final_res['HaveQuasifi']==True]
final_res = final_res[['antecedents', 'consequents', 'support', 'confidence', 'QIrhs']]
for i in range(len(final_res)):
  final_res['antecedents'].iloc[i] = list(final_res.iloc[i]['antecedents'])
  final_res['consequents'].iloc[i] = list(final_res.iloc[i]['consequents'])
  
final_res.to_csv('rule_care.csv', index=False) #SET OF RULE CARE

In [ ]:
# TRANSFORM DATASET D TO OTHER FORM OF VALUE
for i in range(len(df)):
    for j in df.columns:
        df[j].iloc[i]= j+':'+str(df[j].iloc[i])

df.to_csv('transformed_input.csv', index=False)

In [ ]:
#APPLY DATA STRUCTURE FOR RULE CARE AND CALCULATE INITIAL BUDGET FOR EACH RULE
import math
rules = pd.read_csv('./rule_care.csv')
rules_care = [[eval(rules['antecedents'].iloc[i]), eval(rules['consequents'].iloc[i]), rules['support'].iloc[i], rules['confidence'].iloc[i], rules['QIrhs'].iloc[i]] for i in range(len(rules))]

budget = {}
total_rows = len(df)
for r in rules_care:
    rule = (tuple(r[0]+r[1]))
    if r[4]:
        budget[rule]= round(min((r[2]-min_sup)*total_rows, math.floor(r[2]*total_rows*(r[3]-min_conf)/r[3])))
    else:
        budget[rule]= round(min((r[2]-min_sup)*total_rows, math.floor(r[2]*total_rows*(r[3]-min_conf)/(r[3]*(1-min_conf)))))


In [ ]:
#APPLY DATA STRUCTURE FOR DATASET
import collections
df = pd.read_csv('./transformed_input.csv')
subset = df[quasifiIden]
tuples = [tuple(x) for x in subset.to_numpy()]
freq= dict(collections.Counter(tuples)) #GROUPING EACH ROW WITH ITS KEY AS TUPLE OF QSI VALUE

#SAVE THE INDEX OF EACH ROW IN DATA SET ACCORDING TO THEIR GROUP
index_map = {}
for index, row in df.iterrows():
    t = tuple(row[:6])
    if t in index_map:
        index_map[t].append(index)
    else:
        index_map[t] = [index]
        
orgin = {}
current = {}
for i in index_map:
    for j in index_map[i]:
        orgin[j]=i
        current[j]=i


In [ ]:
import random
sg = []
ug = []
ug_small = []
ug_big = []
um = []

#STATE OF EACH GROUP CAN GIVE OR BE GIVEN
state = freq.copy()
for i in state:
    state[i]=0

for i in freq:
    if freq[i] >= k:
        sg.append(i)
    else:
        ug.append(i)
        if freq[i] <= k/2:
            ug_small.append(i)
        else:
            ug_big.append(i)

ug = sorted(ug, key=lambda x: freq[x])
SelfG = None

#COUNT THE AFFECTED RULE OF TUPLE t
def count_Rt(rules, t):
    res = []
    for  r in rules:
        check = True
        for i in r[0]+r[1]:
            if i not in t:
                check = False
                break
        if check:
            res.append(r)
    return res
            

def issafe(g):
    return freq[g]>=k or freq[g] == 0

def risk(m,k):
    # if m == 0:
    #     return 0
    if m >=k:
        return 0
    else:
        return 2*k - m

def check_flow(i, j):
    if i == -1 and j == 1:
        return True
    if i == -1 and j == 0:
        return True
    if i == 0 and j == 1:
        return True
    if i == 0 and j == 0:
        return True
    return False

def check_budget(gi,gj):
    qij = [gi[k] for k in range(len(gi)) if gj[k]!=gi[k]]
    rij= count_Rt(rules_care, qij)
    for r in rij:
            tr = tuple(r[0]+r[1])
            if (budget[tr] <=0):
                return False
    return True

def disperse(selfg):
    for i in index_map[selfg]:
        index_map[orgin[i]].append(i)
        index_map[current[i]].remove(i)
        freq[orgin[i]]+=1
        freq[current[i]]-=1
        if freq[current[i]] == 0:
            um.remove(current[i])
        current[i]=orgin[i]
        if freq[orgin[i]] ==1 and orgin[i] not in um:
            um.append(orgin[i])
    id = random.randint(0, len(sg)-1)
    index_map[sg[id]]+=index_map[selfg]
    index_map[selfg]=[]
    freq[sg[id]]+=freq[selfg]
    freq[selfg]=0

def find_group_to_migrate(selg, rmg):
    # target = None
    # state = 0
    #POLICY
    #1. Nếu 1 group cho tuple thì nó sẽ luôn luôn luôn cho, và ngược lại nhận thì sẽ luôn luôn nhận
    #2. gi -> gj (Vr thuoc Ri,gi->gj, Budget_r > 0)
    #3. Give 2 group gi, gj. Assume gi is k-unsafe group(freq[gi] < k). The number of migrant tuples (mgrtN) is determined:
    #    Case 1: gj is k-unsafe group(freq[gj] < k). If gi->gj, then mgrtN = min(|gi|, k - |gj|). If gi <- gj then mgrtN = min(|gj|, k - |gi|)
    #    Case 2: gj is k-safe group(freq[gj] >= k). If gi->gj, then mgrtN = |gi|. If gi <- gj then mgrtN <= Min(k - |gi|, |gj|-k, |origin(gj)|), 
    #       when min(|origin(gj)|, |gj|-k) = 0 then gi <= gj is impossible.
    
    candidates = None
    max_reduction = -100
    for g in rmg: # UGB + USG + SG -> rate risk_after = 0
        init_risk = risk(freq[selg],k) + risk(freq[g],k)
        
        if not (check_flow(state[selg], state[g]) or (check_flow(state[g], state[selg]))):
            continue
            
        if not issafe(g): # g is k-unsafe group
            # selfg -> g    
            if check_flow(state[selg], state[g]):
                #Check Budget selg -> g
                if not check_budget(selg,g):
                    continue
                
                mgrntN1 = min(freq[selg], k - freq[g])
                after_risk1 = risk(freq[selg] - mgrntN1,k) + risk(freq[g] + mgrntN1,k)
                risk_reduce1 = init_risk - after_risk1
                
                if after_risk1 == 0:
                    return (g, mgrntN1, 1)
                
                if risk_reduce1 > max_reduction:
                    max_reduction = risk_reduce1
                    candidates = (g, mgrntN1, 1)
            
                    
            if check_flow(state[g], state[selg]):
            # selfg <- g 
            
                #Check Budget g -> selfg
                if not check_budget(g,selg):
                    continue

                mgrntN2 = min(freq[selg], k - freq[g])
                after_risk2 = risk(freq[selg] + mgrntN2,k) + risk(freq[g] - mgrntN2,k)
                risk_reduce2 = init_risk - after_risk2
        
                if after_risk2 == 0:
                    return (g, mgrntN2, -1)
                
                if risk_reduce2 > max_reduction:
                    max_reduction = risk_reduce2
                    candidates = (g, mgrntN2, -1)
        else:
            if check_flow(state[g], state[selg]):
                #Check Budget g -> selfg
                if not check_budget(g,selg):
                    continue
                
                if freq[g] == k: 
                    continue
                mgrntN2 = min(k - freq[selg], freq[g]-k)
                after_risk2 = risk(freq[selg] + mgrntN2,k) + risk(freq[g] - mgrntN2,k)
                risk_reduce2 = init_risk - after_risk2
        
                if after_risk2 == 0:
                    return (g, mgrntN2, -1)
                
                if risk_reduce2 > max_reduction:
                    max_reduction = risk_reduce2
                    candidates = (g, mgrntN2, -1)
                    
                    
            if check_flow(state[selg], state[g]):
                if not check_budget(selg,g):
                    continue

                mgrntN1 = freq[SelfG]
                after_risk1 = risk(freq[selg] - mgrntN1,k) + risk(freq[g] + mgrntN1,k)
                risk_reduce1 = init_risk - after_risk1
                
                if after_risk1 == 0:
                    return (g, mgrntN1, 1)
                
                if risk_reduce1 > max_reduction:
                    max_reduction = risk_reduce1
                    candidates = (g, mgrntN1, 1)
                        
   
    return candidates
            
                    
            
    
while len(ug) > 0 or SelfG:
        if SelfG is None:
            SelfG = ug.pop(0)
        if SelfG in ug_small:
            ug_small.remove(SelfG)
        elif SelfG in ug_big:
            ug_big.remove(SelfG)
        if freq[SelfG] <= k/2:
            remaining_groups = ug_big + ug_small + sg
        else:
            remaining_groups = ug_small + ug_big + sg

        g = find_group_to_migrate(SelfG, remaining_groups)
        if g is None:
            um.append(SelfG)
            SelfG = None
            
        else:
            Nmgrt = g[1]
            #Migrate from SelG to G g[1] tuples
            if g[2] == 1:
                freq[SelfG] -= Nmgrt
                freq[g[0]] += Nmgrt
                state[g[0]] = 1
                state[SelfG] = -1
                for i in index_map[SelfG][:Nmgrt]:
                    current[i]=g[0]
                index_map[g[0]]+= index_map[SelfG][:Nmgrt]
                index_map[SelfG] = index_map[SelfG][Nmgrt:]
                Qij = [SelfG[i] for i in range(len(SelfG)) if g[0][i]!=SelfG[i]]
                Ri_j= count_Rt(rules_care, Qij)
                for r in Ri_j:
                    tr = tuple(r[0]+r[1])
                    budget[tr]-= 1
                    
            else:
                freq[SelfG] += Nmgrt
                freq[g[0]] -= Nmgrt
                state[g[0]] = -1
                state[SelfG] = +1
                for i in index_map[g[0]][:Nmgrt]:
                        current[i]=SelfG
                index_map[SelfG]+= index_map[g[0]][:Nmgrt]
                index_map[g[0]] = index_map[g[0]][Nmgrt:]
                Qij = [g[0][i] for i in range(len(g[0])) if g[0][i]!=SelfG[i]]
                Ri_j= count_Rt(rules_care, Qij)
                for r in Ri_j:
                    tr = tuple(r[0]+r[1])
                    budget[tr]-= 1
                
            
            if issafe(SelfG):
                sg.append(SelfG)
                if SelfG in ug:
                    ug.remove(SelfG)
                if SelfG in ug_big:
                    ug_big.remove(SelfG)
                if SelfG in ug_small:
                    ug_small.remove(SelfG)
                state[SelfG] = 0
            
            if issafe(g[0]):
                sg.append(g[0])
                if g[0] in ug:
                    ug.remove(g[0])
                if g[0] in ug_big:
                    ug_big.remove(g[0])
                if g[0] in ug_small:
                    ug_small.remove(g[0])
                state[g[0]] = 0
        
        
            print(SelfG, index_map[SelfG],freq[SelfG])
            print(g[0],index_map[g[0]],freq[g[0]])
            
            if issafe(SelfG) and issafe(g[0]):
                SelfG = None
            else:
                if not issafe(g[0]):
                    SelfG = g[0]
         
if len(um) >0:
    for g in um:
        disperse(g)   

In [ ]:
ds = pd.read_csv('./transformed_input.csv')
for i in index_map:
    for j in index_map[i]:
        for k in range(len(quasifiIden)):
            ds.iloc[j][quasifiIden[k]]=i[k]

ds.to_csv('result.csv', index=False)

In [ ]:
dff = pd.read_csv('result.csv')
for i in range(len(dff)):
    for j in dff.columns:
        val = dff.iloc[i][j].split(':')[1]
        dff.iloc[i][j] = val

        
dff.to_csv('output.csv', index=False)